In [1]:
from selenium import webdriver
import pandas as pd

In [2]:
# Get forum topics
driver = webdriver.Chrome('C:\Program Files (x86)\Google\Chrome\chromedriver')
driver.get('https://www.maintenance.org/forum/posts-about-root-cause-analysis')
driver.implicitly_wait(64)

topics = driver.find_elements_by_class_name('h-topic-extract-title')
topic_names = []
    
for i in topics:
    topic_names.append(i.find_element_by_css_selector('a').get_attribute('href')) 

In [3]:
print(topic_names)

['https://www.maintenance.org/topic/type-of-bearing-failure', 'https://www.maintenance.org/topic/bearing-current-damage-on-nde-bearing-due-to-bearing-current', 'https://www.maintenance.org/topic/is-bearing-having-fluting-marks', 'https://www.maintenance.org/topic/reciprocating-compressor-liner-failure', 'https://www.maintenance.org/topic/gear-failure-at-screw-pump', 'https://www.maintenance.org/topic/vfd-failure', 'https://www.maintenance.org/topic/complet-root-cause-analysis-on-a-wheel-loader', 'https://www.maintenance.org/topic/steam-path-audit', 'https://www.maintenance.org/topic/catastrophic-failure-2', 'https://www.maintenance.org/topic/reference-for-root-cause-analysis', 'https://www.maintenance.org/topic/broken-gear-shaft', 'https://www.maintenance.org/topic/gas-turbine-bearing-failure-lube-oil-leakage-caught-fire', 'https://www.maintenance.org/topic/mechanical-seal-failure-1', 'https://www.maintenance.org/topic/firewater-pumps-pandid-and-network-over-pressure-protection', 'http

In [7]:
def extract_data(comments):
    #xpath //*[@id="post_body_460827630053833370"] 
    posts = driver.find_elements_by_xpath("//*[contains(@id, 'post_body_')]") 
    post_ids = []
        
    for post in posts:
        post_ids.append(post.get_attribute('id')) 
    
    # Remove post_body_ from each id and convert back to string
    post_ids = list(map(lambda sub:int(''.join( 
      [element for element in sub if element.isnumeric()])), post_ids)) 
        
    for x in post_ids:
        try: 
            #Extract Topic
            # xpath //*[@id="pageTitle"]
            page_topic = driver.find_element_by_xpath('//*[@id="pageTitle"]')
            topic = page_topic.text
            
            #Extract Dates from each User in Topic
            # xpath //*[@id="460686892548839591"]/div[1]/div/div[2]/time
            user_date = driver.find_element_by_xpath('//*[@id="'+ str(x) +'"]/div[1]/div/div[2]/time') 
            date = user_date.get_attribute('datetime')

            #Extract User IDs for each User in Topic
            # xpath //*[@id="460686892548839591"]/div[1]/div/div[1]/a
            userid_element = driver.find_element_by_xpath('//*[@id="'+ str(x) +'"]/div[1]/div/div[1]/a')
            userid = userid_element.text
            #topics = driver.find_elements_by_class_name('h-topic-extract-title')
            #topic_names.append(i.find_element_by_css_selector('a').get_attribute('href')) 

            #Extract Comments from each User in Topic
            # xpath //*[@id="post_body_460686892553114974"]
            # xpath //*[@id="post_body_460686892548839591"]/p
            post = ""
            user_message = driver.find_elements_by_xpath('//*[@id="post_body_'+ str(x) +'"]')
            for message in user_message:
                try:
                    post += str(message.text)
                except:
                    continue       
                    
            #Adding date, userid and comment for each user in a dataframe
            comments.loc[len(comments)] = [topic,date,userid,post]
            
        except:
            continue
    return(comments)
            

In [8]:
# Create comments data frame
comments = pd.DataFrame(columns = ['topic','date','userid','post'])

# Crawl through topics to extract comments
for topic in topic_names:
    url = str(topic)
    
    driver = webdriver.Chrome(executable_path='C:\Program Files (x86)\Google\Chrome\chromedriver') 
    try:
        driver.get(url)
        driver.implicitly_wait(64)
        comments = extract_data(comments)   
                
    except:
        driver.quit()
        raise
        
# Save comments to CSV file
comments.to_csv ('comments.csv', index = None, header=True)

driver.quit()
